# 导入模块

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")
import requests
import hmac
import hashlib
import base64
import pymysql
from sqlalchemy import create_engine
import json
import datetime
import time
from datetime import timedelta , datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tomt.reason, tprm.purchase_amount, tprm.liquidated_damages 违约金
, om.order_type, om.union_rent_tag
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id

where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4
-- and  om.create_time = CURRENT_DATE               -- 近15天数据
and  date_format(om.create_time, '%Y-%m-%d') >= '2025-04-12'
'''

df_order = query(sql1)
df_order.shape

(1537707, 47)

In [5]:
sql_risk = ''' -- risk等级
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_tag') end,'"','') as union_rent_tag
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_rejected') end,'"','') as union_rent_rejected

from db_credit.risk
'''

df_risk = query(sql_risk)

In [6]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

In [7]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

# 提取订单风控数据

# 合并数据

In [8]:
df = df_order.copy()
df = df[df.type!=4]
df.order_id.notnull().sum()

np.int64(1537707)

# 处理日期

In [9]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year
df['hour'] = df['create_time'].dt.hour
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

# 渠道补充

In [10]:
# qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
# qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)
# qudao_name_df.rename(columns={'订单号': '订单号1'}, inplace=True)
# df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号1',how = 'left')
# df["channel_name"] = np.where(df.渠道名称.notnull(),df.渠道名称,df["channel_name"])

FileNotFoundError: [Errno 2] No such file or directory: 'F:/myfile/租后数据/历史订单渠道名称补充.xlsx'

# 处理备注信息

In [11]:
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  


def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]

df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()


df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

# 渠道归属

In [12]:

df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

# 订单去重

In [13]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# 定义状态

In [14]:
# note 2025-02-24添加
df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')
# , 'union_rent_tag', 'union_rent_rejected'
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
# note 2025-02-24修改
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

In [15]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1) 
# df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截']),axis=1) 

# 保留商家的数据

In [16]:
df_contain = df.copy()

# 剔除商家数据

In [17]:
# 调用Data_Clean类中的drop_merchant函数
df = clean.drop_merchant(df)

# 去重订单明细导出

In [18]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date = pd.to_datetime(t_date) - timedelta(days=2)
t_date

Timestamp('2025-07-27 00:00:00')

# 导入服务订单

In [19]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

df_j = df[df["进件"]=="进件"]
df_j.shape

(175990, 76)

In [20]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)

df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)


df['是否出库'] = np.where((df['人审拒绝']==0)&(df['客户取消']==0)&(df['无法联系']==0)&(df['待审核']==0)&(df['是否出库']==1), 1, 0)
df["出库前风控强拒"] = np.where((df["审核状态"]=='出库前风控强拒')&(df['是否出库']==0),1,0)
df["机审强拒"] = np.where((df["审核状态"]=='机审强拒')&(df['是否出库']==0),1,0)
df['机审通过件'] = np.where((df['是否进件']==1)&(df['机审强拒']==0), 1, 0)
df['风控通过件'] = np.where((df['是否进件']==1)&(df['机审强拒']==0)&(df['人审拒绝']==0), 1, 0)

df['已退款'] = np.where((df['风控通过件']==1)&(df['审核状态']=='已退款'), 1, 0)

df['是否二手'] = np.where(df['product_name'].str.contains(r'99新|95新|准新|90新'), 1, 0)

# 总体转化

## 剔除拒量订单

In [21]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)

# 支付宝联合运营

In [130]:
# union_rent_tag是否为支付宝联合运营的标记，y是，n不是默认不是
df_lhyy = df[df.union_rent_tag=='Y']
df2_lhyy = df2[df2.union_rent_tag=='Y']
df_group = all_models.data_group(df_lhyy, df2_lhyy, df_risk_examine, '下单日期')
df_group = df_group[["进件数","机审强拒","人审拒绝","客户取消","无法联系","出库前风控强拒","待审核",'出库']]
df_group = df_group.iloc[-1, :]
df_group
jj = df_group.loc['进件数']
qx = df_group.loc['机审强拒']+df_group.loc['人审拒绝']+df_group.loc['客户取消']+df_group.loc['无法联系']+df_group.loc['出库前风控强拒']
sh = df_group.loc['待审核']
tg = df_group.loc['出库']


# 各位好，今天进件5单，客户取消0单， 待审核3单，通过2单
if jj!=0:
    info = f'''各位好，今天进件{jj}单，客户取消{int(qx)}单，待审核{sh}单，通过{tg}单'''
else:
    info = '各位好，今天没有进件'
info

'各位好，今天进件2.0单，客户取消1单，待审核1.0单，通过0.0单'

In [131]:
df_group

进件数        2.0
机审强拒       0.0
人审拒绝       0.0
客户取消       1.0
无法联系       0.0
出库前风控强拒    0.0
待审核        1.0
出库         0.0
Name: 2025-04-14 00:00:00, dtype: float64

# 钉钉机器人

In [133]:
def send_dingtalk_message(webhook, secret, message):
    # 计算签名（如果有设置）
    if secret:
        timestamp = str(round(time.time() * 1000))
        string_to_sign = '{}\n{}'.format(timestamp, secret)
        hmac_code = hmac.new(secret.encode('utf-8'), string_to_sign.encode('utf-8'), digestmod=hashlib.sha256).digest()
        sign = base64.b64encode(hmac_code).decode('utf-8')
        webhook = f'{webhook}&timestamp={timestamp}&sign={sign}'

    # 构造消息体
    data = {
        "msgtype": "text",
        "text": {
            "content": message
        }
    }

    # 发送请求
    try:
        response = requests.post(webhook, json=data)
        response.raise_for_status()
        print("消息发送成功")
    except requests.exceptions.RequestException as e:
        print(f"消息发送失败: {e}")


if __name__ == "__main__":
    # 替换为你的 Webhook 地址
    webhook = "https://oapi.dingtalk.com/robot/send?access_token=855fc4879198e837aef0ed789c24963fb5b7f4e613d13dbcf0b99930bd034857"
    # https://oapi.dingtalk.com/robot/send?access_token=85b55eb8850e4506c0145c0249d7465cf787a4332e9829c26d3db55329bac3a7
    # 替换为你的密钥，如果没有设置则留空
    secret = "SEC198d68297672e23239c0fa0531f038bd2fb318c381a1ab898d5dc51fe1374fbb"
    # SECc6b1496b6ba837fa20ef46b93d7788c561c4b8a9dec206f838642f3b6fafd78a
    # 要发送的消息内容
    message = 'https://docs.qq.com/sheet/DV05jVENUY1ZnQm1t?nlc=1&tab=000001'
    send_dingtalk_message(webhook, secret, message)

消息发送成功


In [ ]:
import requests
import json

# 获取 access_token
def get_access_token(appid, appsecret):
    url = f"https://open.docs.qq.com/openapi/token?appid={appid}&appsecret={appsecret}"
    response = requests.get(url)
    result = response.json()
    if result.get("errcode") == 0:
        return result.get("access_token")
    else:
        print(f"获取 access_token 失败: {result.get('errmsg')}")
        return None

# 上传 Excel 文件
def upload_excel_file(access_token, file_path):
    url = "https://open.docs.qq.com/openapi/file/upload"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    files = {'file': open(file_path, 'rb')}
    data = {
        "file_type": 1  # 1 表示 Excel 文件
    }
    response = requests.post(url, headers=headers, files=files, data=data)
    result = response.json()
    if result.get("errcode") == 0:
        return result.get("docid")
    else:
        print(f"上传文件失败: {result.get('errmsg')}")
        return None

# 修改文档权限为所有人可编辑
def set_doc_permission(access_token, docid):
    url = "https://open.docs.qq.com/openapi/doc/permission"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    data = {
        "docid": docid,
        "permission": 3  # 3 表示所有人可编辑
    }
    response = requests.post(url, headers=headers, json=data)
    result = response.json()
    if result.get("errcode") == 0:
        print("文档权限修改成功")
    else:
        print(f"文档权限修改失败: {result.get('errmsg')}")

if __name__ == "__main__":
    appid = "your_appid"
    appsecret = "your_appsecret"
    file_path = "your_excel_file.xlsx"

    # 获取 access_token
    access_token = get_access_token(appid, appsecret)
    if access_token:
        # 上传 Excel 文件
        docid = upload_excel_file(access_token, file_path)
        if docid:
            # 修改文档权限
            set_doc_permission(access_token, docid)
